In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import torch
import math
from torch.nn.utils import prune
import tqdm.notebook as tqdm
from functools import partial
import torchvision
from collections import OrderedDict
import pandas as pd
from scipy.stats import norm
import random
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

def set_global_seed(seed: int) -> None:
    """
    Set global seed for reproducibility.
    """

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_global_seed(42)

## Загрузка и обработка данных

In [ ]:
import torchvision


In [ ]:
transform = torchvision.transforms.Compose([
    # torchvision.transforms.Resize(size=(224, 224)),
    # torchvision.transforms.ToTensor(),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomCrop(32, 4),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=(0.4914, 0.4822, 0.4465),
        std=(0.2023, 0.1994, 0.2010)
    )
])

In [ ]:
ds_train = torchvision.datasets.CIFAR10(
    root='./', train=True, transform=transform, download=True
)
ds_test = torchvision.datasets.CIFAR10(
    root='./', train=False,
    transform=torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=(0.4914, 0.4822, 0.4465),
        std=(0.2023, 0.1994, 0.2010)
    )
    ]),
    download=True
)
ds_train, ds_test

Transfom to PIL image

In [ ]:
tmean, tstd = transform.transforms[-1].mean, transform.transforms[-1].std
tmean, tstd = np.array(tmean), np.array(tstd)

inverse_transform = torchvision.transforms.Compose([
    torchvision.transforms.Normalize(
        mean=-tmean / tstd,
        std=1.0 / tstd
    ),
    torchvision.transforms.ToPILImage()
])

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(13, 6))

for idx, ds in enumerate((ds_train, ds_test)):
    for jdx, kdx in enumerate(np.random.randint(0, len(ds), size=5)):
        image, label = ds[kdx]
        axes[idx, jdx].imshow(inverse_transform(image))
        axes[idx, jdx].set_title(f'Метка: {label} -> {ds.classes[label]}')

axes[0, 0].set_ylabel('Обучающая выборка')
axes[1, 0].set_ylabel('Тестовая выборка')

fig.tight_layout()
plt.show()

In [ ]:
BATCH_SIZE = 128
DEVICE = torch.device('cpu')
if torch.cuda.is_available():
    DEVICE = torch.device('cuda', 0)

print(type(DEVICE), DEVICE)

In [ ]:
dl_train = torch.utils.data.DataLoader(
    dataset=ds_train, batch_size=BATCH_SIZE,
    num_workers=2, shuffle=True
)
dl_test = torch.utils.data.DataLoader(
    dataset=ds_test, batch_size=BATCH_SIZE,
    num_workers=2, shuffle=False
)

In [ ]:
class BasicBlock(torch.nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = torch.nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = torch.nn.BatchNorm2d(planes)
        self.conv2 = torch.nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = torch.nn.BatchNorm2d(planes)

        self.shortcut = torch.nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = torch.nn.Sequential(
                torch.nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                torch.nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(torch.nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = torch.nn.BatchNorm2d(planes)
        self.conv2 = torch.nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = torch.nn.BatchNorm2d(planes)
        self.conv3 = torch.nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = torch.nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = torch.nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = torch.nn.Sequential(
                torch.nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                torch.nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(torch.nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = torch.nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = torch.nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return torch.nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])


def test():
    net = ResNet18()
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())

test()

In [ ]:
import tqdm.notebook as tqdm
from functools import partial
from torch.utils.tensorboard import SummaryWriter


In [ ]:
def count_parameters(net) -> int:
  count_params = 0
  count_zero_params = 0
  for name, module in net.named_modules():
      if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
          count_params += module.weight.numel()
          count_zero_params += torch.sum(module.weight == 0)
          sparsity = 100. * float(torch.sum(module.weight == 0)) / float(module.weight.nelement())
          print(f'Module len: {len(module.weight)}; Sparsity in {name}.weight with {module.weight.numel()} parameters: {sparsity:0.3f}%')
  print(f'Global sparsity: {100. * (float(count_zero_params) / float(count_params)):0.3f}%')
  return count_params

In [ ]:
def test_model(model, train_dataloader) -> float:
    model.to(DEVICE)
    model.eval()
    with torch.no_grad():
        train_accuracies = []
        for images, labels in tqdm.tqdm(train_dataloader, total=len(train_dataloader), leave=False):
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs  = model(images)
            train_accuracies.append(torch.sum(outputs.argmax(dim=1) == labels)/labels.shape[0])
        return float((sum(train_accuracies) / len(train_accuracies)).cpu() * 100)

In [ ]:
class Relevance():
    def __init__(self, model, dummy_percent, n_percent, m_percent) -> None:
        self.n_percent = n_percent
        self.module_size_list = []
        self.n_list = []
        self.dummy_size_list = []
        self.m_list = []
        self.r_list = []
        self.q_list = []

        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Conv2d):
                module_size = module.weight.shape[0]
                self.module_size_list.append(module_size)
                self.n_list.append(round(module_size * (n_percent / 100)))
                dummy_size = round(module_size * (dummy_percent / 100))
                self.dummy_size_list.append(dummy_size)
                self.m_list.append(round(dummy_size * (m_percent / 100)))

                self.r_list.append(torch.zeros(module_size))
                self.q_list.append(torch.zeros(dummy_size))
        self.criterion_value_list = [0]

        self.feature_indexes_list = [np.arange(n) for n in self.module_size_list]
        self.feature_mask_list = []
        self.dummy_feature_mask_list = []

    def _del_mask(self, model):
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Conv2d):
                if prune.is_pruned(module):
                    with torch.no_grad():
                        self._tensor_name = 'weight'
                        if hasattr(module, self._tensor_name):
                            delattr(module, self._tensor_name)
                        orig = module._parameters[self._tensor_name + "_orig"]
                        del module._parameters[self._tensor_name + "_orig"]
                        del module._buffers[self._tensor_name + "_mask"]
                        module._forward_pre_hooks = OrderedDict()
                        setattr(module, self._tensor_name, orig)
        self.feature_mask_list = []
        torch.cuda.empty_cache()

    def update_mask(self, model):
        feature_mask_list = []
        dummy_feature_mask_list = []
        layer_num = 0
        self._del_mask(model)
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Conv2d):
                feature_indexes_sample = random.sample(list(self.feature_indexes_list[layer_num]), self.n_list[layer_num])
                feature_mask = torch.zeros(self.module_size_list[layer_num])
                feature_mask[self.feature_indexes_list[layer_num]] = 1
                feature_mask[feature_indexes_sample] = 0
                feature_mask_list.append(feature_mask)

                dummy_feature_indexes = np.arange(self.dummy_size_list[layer_num])
                dummy_feature_indexes_sample = random.sample(list(dummy_feature_indexes), self.m_list[layer_num])
                dummy_feature_mask = torch.ones(self.dummy_size_list[layer_num])
                dummy_feature_mask[dummy_feature_indexes_sample] = 0
                dummy_feature_mask_list.append(dummy_feature_mask)
                layer_num += 1
        self.feature_mask_list = feature_mask_list

        self.dummy_feature_mask_list = dummy_feature_mask_list
        torch.cuda.empty_cache()

    def apply_mask(self, model):
        layer_num = 0
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Conv2d):
                if (self.feature_mask_list):
                    weight_mask = torch.ones_like(module.weight)
                    weight_mask[~torch.tensor(self.feature_mask_list[layer_num], dtype=bool)] = 0
                    prune.custom_from_mask(module=module, name='weight', mask=weight_mask)
                elif (self.feature_indexes_list):
                    feature_mask = torch.zeros(self.module_size_list[layer_num])
                    feature_mask[self.feature_indexes_list[layer_num]] = 1
                    weight_mask = torch.ones_like(module.weight)
                    weight_mask[~torch.tensor(feature_mask, dtype=bool)] = 0
                    prune.custom_from_mask(module=module, name='weight', mask=weight_mask)
                layer_num += 1
        torch.cuda.empty_cache()

    def update_relevance(self, criterion_value):
        for i in range(len(self.r_list)):
            cur_delta = (criterion_value - np.mean(self.criterion_value_list))
            self.r_list[i] += cur_delta * self.feature_mask_list[i].flatten()
            self.q_list[i] += cur_delta * self.dummy_feature_mask_list[i].flatten()

        self.criterion_value_list.append(criterion_value)
        torch.cuda.empty_cache()

    def cut_by_probability(self):
        module_count = len(self.r_list)
        for i in range(module_count):
            mu = float(torch.mean(self.q_list[i]))
            sigma = float(torch.std(self.q_list[i]))
            probability_list = [norm.cdf(r, mu, sigma) for r in self.r_list[i][self.feature_indexes_list[i]]]
            prune_indexes = np.argsort(probability_list)[:self.n_list[i]]
            print(len(prune_indexes))
            self.feature_indexes_list[i] = np.array([idx for idx in self.feature_indexes_list[i] if idx not in self.feature_indexes_list[i][prune_indexes]])

            self.r_list[i] = torch.zeros(self.module_size_list[i])
            self.q_list[i] = torch.zeros(self.dummy_size_list[i])
        torch.cuda.empty_cache()




In [ ]:
def training_loop(n_epochs, network, loss_fn, optimizer,scheduler, dl_train, dl_test, device):
    '''
    :param int n_epochs: Число итераций оптимизации
    :param torch.nn.Module network: Нейронная сеть
    :param Callable loss_fn: Функция потерь
    :param torch.nn.Optimizer optimizer: Оптимизатор
    :param torch.utils.data.DataLoader dl_train: Даталоадер для обучающей выборки
    :param torch.utils.data.DataLoader dl_test: Даталоадер для тестовой выборки
    :param torch.Device device: Устройство на котором будут происходить вычисления
    :returns: Списки значений функции потерь и точности на обучающей и тестовой выборках после каждой итерации
    '''
    loss_fn.to(device)
    train_losses, test_losses, train_accuracies, test_accuracies = [], [], [], []
    pbar = tqdm.tqdm(range(n_epochs), total=n_epochs, leave=False)
    for epoch in (pbar):
        # Переводим сеть в режим обучения
        network.train()

        # Итерация обучения сети
        for batch_idx, (images, labels) in enumerate(tqdm.tqdm(dl_train, total=len(dl_train), leave=False)):
            images = images.to(device)
            labels = labels.to(device)

            global_idx = (epoch * len(dl_train) + batch_idx)

            if global_idx < 21896 and global_idx %  test_relevance.n_percent == 0:
                test_relevance.update_mask(network)
            else:
                test_relevance._del_mask(network)
            test_relevance.apply_mask(network)

            optimizer.zero_grad()
            loss = loss_fn(network(images), labels)
            loss.backward()
            optimizer.step()
            scheduler.step()

            if global_idx < 21896 and global_idx % test_relevance.n_percent == 0:
                test_relevance.update_relevance(float(loss))

            if global_idx != 0 and global_idx % 5474 == 0:
                print('!' * 100)
                test_relevance._del_mask(network)
                count_parameters(network)
                test_relevance.cut_by_probability()
                test_relevance.apply_mask(network)
                count_parameters(network)
                print('!' * 100)

            torch.cuda.empty_cache()

        # Оцениваем качество модели каждые 3 итерации
        if epoch % 3 == 0 or epoch == n_epochs - 1:
            # Переводим сеть в инференс режим
            network.eval()

            # При тестировании сети нет необходимости считать градиенты, поэтому можно отключить автоматическое дифференцирование
            #   для ускорения операций
            with torch.no_grad():
                # Вычисление качества и функции потерь на обучающей выборке
                tmp_train_losses, tmp_train_accuracies = [], []
                for images, labels in tqdm.tqdm(dl_train, total=len(dl_train), leave=False):
                    images = images.to(device)
                    labels = labels.to(device)

                    test_relevance.apply_mask(network)
                    outputs  = network(images)
                    test_relevance._del_mask(network)

                    tmp_train_losses.append(loss_fn(outputs, labels))
                    tmp_train_accuracies.append(torch.sum(outputs.argmax(dim=1) == labels)/labels.shape[0])

                train_losses.append((sum(tmp_train_losses) / len(tmp_train_losses)).cpu())
                train_accuracies.append((sum(tmp_train_accuracies) / len(tmp_train_accuracies)).cpu() * 100)
                # Вычисление качества и функции потерь на тестовой выборке
                tmp_test_losses, tmp_test_accuracies = [], []
                for images, labels in tqdm.tqdm(dl_test, total=len(dl_test), leave=False):
                    images = images.to(device)
                    labels = labels.to(device)

                    test_relevance.apply_mask(network)
                    outputs  = network(images)
                    test_relevance._del_mask(network)

                    tmp_test_losses.append(loss_fn(outputs, labels))
                    tmp_test_accuracies.append(torch.sum(outputs.argmax(dim=1) == labels)/labels.shape[0])

                test_losses.append((sum(tmp_test_losses) / len(tmp_test_losses)).cpu())
                test_accuracies.append((sum(tmp_test_accuracies) / len(tmp_test_accuracies)).cpu() * 100)

            pbar.set_description(
                'Loss (Train/Test): {0:.3f}/{1:.3f}. Accuracy, % (Train/Test): {2:.2f}/{3:.2f}\n'.format(
                    train_losses[-1], test_losses[-1], train_accuracies[-1], test_accuracies[-1]
                )
            )

    return train_losses, test_losses, train_accuracies, test_accuracies

## 20%

In [ ]:
N_PERCENT = 5
M_PERCENT = 10
DUMMY_PARAMS_PERCENT = 90

train_func = partial(
    training_loop, n_epochs=70, loss_fn=torch.nn.CrossEntropyLoss(),
    dl_train=dl_train, dl_test=dl_test, device=DEVICE
)

In [ ]:
conv_net = ResNet18()
conv_net.to(DEVICE)

optimizer = torch.optim.SGD(conv_net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.9)

test_relevance = Relevance(conv_net, DUMMY_PARAMS_PERCENT, N_PERCENT, M_PERCENT)

In [ ]:
train_losses, test_losses, train_accs, test_accs = train_func(
    network=conv_net,
    optimizer=optimizer,
    scheduler=scheduler
)

In [ ]:
import time
np.save(GLOBAL_PATH + f'/ResNet_train_accuracy_{time.strftime("%d.%m.%Y-%H:%M")}.npy', train_accs)
np.save(GLOBAL_PATH + f'/ResNet_test_accuracy_{time.strftime("%d.%m.%Y-%H:%M")}.npy', test_accs)

In [ ]:
test_relevance.apply_mask(conv_net)
test_model(conv_net, dl_test)

In [ ]:
test_relevance.apply_mask(conv_net)
count_parameters(conv_net)

In [ ]:
SMALL_SIZE = 16
MEDIUM_SIZE = 10
BIGGER_SIZE = 8
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


fig, ax = plt.subplots(1, 1, figsize=(7, 4))
ax.plot(np.arange(len(train_accs)) * 3, train_accs, label="Точность на обучении", color='red', marker='.', linestyle='-.')
ax.plot(np.arange(len(test_accs)) * 3, test_accs, label="Точность на тесте", color='red', marker='*')

ax.set_xlabel("Номер эпохи")
ax.set_ylabel("Точность, %")

ax.grid(True)
ax.legend(loc='lower right')

fig.tight_layout()
plt.show()

## 40%

In [ ]:
N_PERCENT = 10
M_PERCENT = 10
DUMMY_PARAMS_PERCENT = 90

train_func = partial(
    training_loop, n_epochs=70, loss_fn=torch.nn.CrossEntropyLoss(),
    dl_train=dl_train, dl_test=dl_test, device=DEVICE
)

In [ ]:
conv_net = ResNet18()
conv_net.to(DEVICE)

optimizer = torch.optim.SGD(conv_net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.95)

test_relevance = Relevance(conv_net, DUMMY_PARAMS_PERCENT, N_PERCENT, M_PERCENT)

In [ ]:
train_losses, test_losses, train_accs, test_accs = train_func(
    network=conv_net,
    optimizer=optimizer,
    scheduler=scheduler
)

In [ ]:
import time
np.save(GLOBAL_PATH + f'/ResNet_train_accuracy_prune_40_{time.strftime("%d.%m.%Y-%H:%M")}.npy', train_accs)
np.save(GLOBAL_PATH + f'/ResNet_test_accuracy_prune_40_{time.strftime("%d.%m.%Y-%H:%M")}.npy', test_accs)

In [ ]:
test_relevance.apply_mask(conv_net)
test_model(conv_net, dl_test)

In [ ]:
SMALL_SIZE = 16
MEDIUM_SIZE = 10
BIGGER_SIZE = 8
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


fig, ax = plt.subplots(1, 1, figsize=(7, 4))
ax.plot(np.arange(len(train_accs)) * 3, train_accs, label="Точность на обучении", color='red', marker='.', linestyle='-.')
ax.plot(np.arange(len(test_accs)) * 3, test_accs, label="Точность на тесте", color='red', marker='*')

ax.set_xlabel("Номер эпохи")
ax.set_ylabel("Точность, %")

ax.grid(True)
ax.legend(loc='lower right')

fig.tight_layout()
plt.show()